In [1]:
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [2]:
data_folder = 'ml_sec_hw_data/data'

In [40]:
def test_binary_loader(bytes, s):
    l = len(bytes)
    print("bytes length: ", l)

    if l == s:
        # Ha a fájl mérete pontosan s
        x = bytes
    elif l < s:
        # Ha a fájl mérete kisebb mint s, hozzáadunk nullás paddingot
        x = np.pad(bytes, (0, s - l), 'constant', constant_values=(0,))
    else:
        # Ha a fájl mérete nagyobb mint s, átlagoljuk a bájtokat
        group_size = np.ceil(l / s) # csoportok számának kiszámítása
        print("group size: ", group_size)
        # Először hozzáadunk szükséges paddingot
        padding_length = s * group_size - l 
        print("padding length: ", padding_length)
        padded_bytes = np.pad(bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
        print("padding bytes: ", padded_bytes)
        # Átlagolás
        reshaped_bytes = padded_bytes.reshape(-1, int(group_size))
        print("reshaped bytes: \n", reshaped_bytes)
        x = np.mean(reshaped_bytes, axis=1)

    return x

In [28]:
x = test_binary_loader([1,2,3,4], 3)
print(x.shape)
print(x)

group size:  2.0
padding length:  2.0
padding bytes:  [1 2 3 4 0 0]
reshaped bytes:  [[1 2]
 [3 4]
 [0 0]]
(3,)
[1.5 3.5 0. ]


In [5]:
tensor_x = torch.tensor(x)
print(tensor_x)

tensor_x = tensor_x.unsqueeze(0)
print(tensor_x.shape)

tensor([1.5000, 3.5000, 0.0000], dtype=torch.float64)
torch.Size([1, 3])


In [114]:
def convert_label(label):
    return 'malware' if label == 1 else 'benign'

In [6]:
class MalwareDataset(Dataset):
    def __init__(self, data_folder, s=2**14):
        self.data_folder = data_folder
        self.data = []
        self.labels = []
        for label in os.listdir(data_folder): # data/.../train -> malware, benign
            for file in os.listdir(os.path.join(data_folder, label)):
                file_path = os.path.join(data_folder, label, file)
                x = self.preprocess_binary_file(file_path, s)
                self.data.append(x)
                self.labels.append(label)

    def preprocess_binary_file(self, file_path, s):
        # Bináris fájl beolvasása
        with open(file_path, 'rb') as file:
            file_bytes = np.fromfile(file, dtype=np.uint8)

        l = len(file_bytes)

        if l == s:
            # Ha a fájl mérete pontosan s
            x = file_bytes
        elif l < s:
            # Ha a fájl mérete kisebb mint s, hozzáadunk nullás paddingot
            x = np.pad(file_bytes, (0, s - l), 'constant', constant_values=(0,))
        else:
            # Ha a fájl mérete nagyobb mint s, átlagoljuk a bájtokat
            # Először hozzáadunk szükséges paddingot
            padding_length = s * np.ceil(l / s) - l
            padded_bytes = np.pad(file_bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
            # Átlagolás
            x = np.mean(padded_bytes.reshape(-1, int(np.ceil(l / s))), axis=1)

        # Normalizálás [0, 1] tartományba
        x_normalized = x / 255

        return x_normalized

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor([1 if self.labels[idx] == 'malware' else 0])
    
def get_loader(data_folder, batch_size=32):
    dataset = MalwareDataset(data_folder)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
train_loader = get_loader(os.path.join(data_folder, 'train'), 32)
test_loader = get_loader(os.path.join(data_folder, 'test'), 32)

In [8]:
data, label = next(iter(train_loader))
print(data.unsqueeze(1).float().shape)
print(label.float().shape)

torch.Size([32, 1, 16384])
torch.Size([32, 1])


In [9]:
class MalwareDetector(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=10, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False),
        )
        self.linear = nn.Linear(in_features=130976, out_features=1, bias=True)
        self.out = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1) # Flatten
        x = self.linear(x)
        x = self.out(x)
        return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MalwareDetector().to(device)
bce = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
model.to(device)

def train(model, train_loader, opt, loss_fn, epochs=10):
    for epoch in range(epochs):
        for data, labels in train_loader:
            data = data.to(device)
            labels = labels.to(device)

            opt.zero_grad()
            out = model(data.unsqueeze(1).float())
            loss = loss_fn(out, labels.float())
            loss.backward()
            opt.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

train(model, train_loader, opt, bce, epochs=10)

Epoch 1/10, Loss: 0.1271088421344757
Epoch 2/10, Loss: 0.0075139193795621395
Epoch 3/10, Loss: 0.004727795720100403
Epoch 4/10, Loss: 0.005919822491705418
Epoch 5/10, Loss: 0.028740501031279564
Epoch 6/10, Loss: 0.002325571607798338
Epoch 7/10, Loss: 0.0006117887678556144
Epoch 8/10, Loss: 0.0008404992404393852
Epoch 9/10, Loss: 0.0007017732714302838
Epoch 10/10, Loss: 0.0009823349537327886


In [249]:
from sklearn.metrics import roc_auc_score, confusion_matrix

def test_model(model, test_loader):
    model.eval()  # Teszt módba állítjuk a modellt
    predictions, true_labels = [], []
    model.to('cpu')
    with torch.no_grad():  # Gradiensek számításának kikapcsolása
        for data, labels in test_loader:
            output = model(data.unsqueeze(1).float())  # Modell alkalmazása
            pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
            predictions.extend(pred_prob)
            true_labels.extend(labels.numpy())
    
    # Bináris osztályozási döntések (0.5 küszöbértékkel)
    pred_labels = [1 if p >= 0.5 else 0 for p in predictions]
    
    # Metrikák kiszámítása
    auc_score = roc_auc_score(true_labels, predictions)
    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels).ravel()
    
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    fpr = fp / (tn + fp)  # False Positive Rate
    fnr = fn / (tp + fn)  # False Negative Rate
    accuracy = (tp + tn) / (tp + tn + fp + fn)  # Átlagos pontosság
    
    print(f"Accuracy: {accuracy}")
    print(f"TPR: {tpr}")
    print(f"TNR: {tnr}")
    print(f"FPR: {fpr}")
    print(f"FNR: {fnr}")
    print(f"AUC: {auc_score}")
    
    return accuracy, tpr, tnr, fpr, fnr, auc_score


accuracy, tpr, tnr, fpr, fnr, auc = test_model(model, test_loader)

Accuracy: 0.9974358974358974
TPR: 0.9948717948717949
TNR: 1.0
FPR: 0.0
FNR: 0.005128205128205128
AUC: 0.9993951347797502


### Kérdések:
1. Mi a betanított modell átlagos pontossága a teszt adaton ha s = 2
14? : 99.48%
2. Mi a modell TPR, TNR, FPR, FNR, valamint AUC értéke a teszt adaton? Mi állapítható meg ezekből a modell
teljesítményéről? : Magas pontosság, nagyon ritkán téveszt a detektorunk. Felismeri mind a malware-t mind a benign-t. Szinte nincs félreosztályzás


In [44]:
original_bytes = [1,2,3,4,5,6,7,8,9,10]
adv_suffix = [0] * 5 # ha egy csoport csakis suffix-t tartalmaz akkor a tömörített verzióban is azok lesznek, azaz módosítani tudjuk őket.
adv_bytes = original_bytes + adv_suffix
print("adv_bytes: ", adv_bytes)
x = test_binary_loader(adv_bytes, 6)
print("input: ", x)

adv_bytes:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0]
bytes length:  15
group size:  3.0
padding length:  3.0
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
input:  [2.         5.         8.         3.33333333 0.         0.        ]


In [65]:
def test_determine_modifiable_positions(bytes, s, suffix_length):
    l = len(bytes)
    modifiable_positions = set()
    original_bytes_lenght = l-suffix_length
    print("bytes length: ", l)
    print("original bytes lenght: ", original_bytes_lenght)

    if l >= s:
        # Számítjuk, hány bájt kerül átlagolásra egy vektor elem létrehozásához
        group_size = int(np.ceil(l / s))
        
        print("group size: ", group_size)
       

        # mennyi group kell az eredeti fájlhoz
        original_groups = int(np.ceil(original_bytes_lenght / group_size))
        print("original groups: ", original_groups) 

        # Meghatározzuk, hány bájt kerül hozzáadásra paddingként
        padding_length = int(s * group_size - l)
        print("padding length: ", padding_length)

        padded_bytes = np.pad(bytes, (0, padding_length), 'constant', constant_values=(0,))
        print("padding bytes: ", padded_bytes)
        
        reshaped_bytes = padded_bytes.reshape(-1, group_size)
        
        # Csoportok tartalmazhatnak: (ha márcsak egy másmilyen elem is van mint suffix, akkor az egész csoport nem módosítható)
            # eredeti fájl csoportok
            # (tiszta) suffix csoportok
            # padding csoportok
        
        print("reshaped bytes: \n", reshaped_bytes)

        # mennyi group kell az eredeti fájlhoz
        padding_groups = int(np.ceil(padding_length / group_size))
        print("padding groups: ", padding_groups) 

        total_length = l+padding_length
        total_groups = int(np.ceil(total_length / group_size))
        print("total groups: ", total_groups)

        for i in range(original_groups+1, total_groups-padding_groups+1):
            print(i)
            modifiable_positions.add(i)

        # # Ha a suffix hossza nagyobb, mint a szükséges padding hossza, az utolsó csoportba eső bájtok módosíthatók
        # if suffix_length > padding_length:
        #     modifiable_positions.add(s - 1)  # Az utolsó vektor pozíció módosítható
    
    return modifiable_positions

# Tesztesetek a függvény ellenőrzésére
# Példa bináris fájlok és az 's' paraméter
binaries = [
    ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10] + [0]*4, 6, 4),  # 4 bájt hozzáadva
    ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10] + [0]*5, 6, 5)   # 5 bájt hozzáadva
]

# A determine_modifiable_positions függvény tesztelése
for binary, s, suffix in binaries:
    modifiable_positions = test_determine_modifiable_positions(np.array(binary), s, suffix)
    print(f"Binary length: {len(binary)}, 's' value: {s}, modifiable positions: {modifiable_positions}")


bytes length:  14
original bytes lenght:  10
group size:  3
original groups:  4
padding length:  4
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
padding groups:  2
total groups:  6
Binary length: 14, 's' value: 6, modifiable positions: set()
bytes length:  15
original bytes lenght:  10
group size:  3
original groups:  4
padding length:  3
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
padding groups:  1
total groups:  6
5
Binary length: 15, 's' value: 6, modifiable positions: {5}


In [183]:
import torch
import numpy as np

def determine_modifiable_positions(binary_tensor, s, suffix_length):
    modifiable_positions = set()

    # Original binary length
    original_binary_length = binary_tensor.shape[0]

    # Attach suffix to binary
    suffix_tensor = torch.zeros(suffix_length, dtype=binary_tensor.dtype)
    binary_suffix = torch.cat((binary_tensor, suffix_tensor), dim=0)

    l = binary_suffix.shape[0]
    
    if l >= s:
        group_size = int(np.ceil(l / s))
        
        # How many groups are needed for the original file
        original_groups = int(np.ceil(original_binary_length / group_size))

        # Calculate the padding length needed
        padding_length = int(s * group_size - l)
        
        # How many groups are needed for the padding
        padding_groups = int(np.ceil(padding_length / group_size))
    
        total_length = l + padding_length
        total_groups = int(np.ceil(total_length / group_size))
        
        # Identify modifiable groups
        for i in range(original_groups + 1, total_groups - padding_groups + 1):
            modifiable_positions.add(i)
    
    return modifiable_positions, len(modifiable_positions), binary_suffix

# # Example Usage:
# data = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32)  # Example binary data tensor
# s = 6  # Target vector length
# suffix_length = 5  # Suffix length

# modifiable_positions, num_modifiable_positions, modified_data = determine_modifiable_positions(data, s, suffix_length)
# print(f"Modifiable positions: {modifiable_positions}, \nNumber of modifiable positions: {num_modifiable_positions}")


In [242]:
victim_loader = get_loader(os.path.join(data_folder, 'victim'), 1)
data, label = next(iter(victim_loader))
print(data.shape)

torch.Size([1, 16384])


In [224]:
attackable_files = []
victim_malware_path = 'ml_sec_hw_data/data/victim/malware'
for file in os.listdir(victim_malware_path):
    file_path = os.path.join(victim_malware_path, file)
    with open(file_path, 'rb') as file:
        file_bytes = np.fromfile(file, dtype=np.uint8)
        if len(file_bytes) >= 2**14:
            attackable_files.append(file_bytes)

In [233]:
test_malware = attackable_files[0]
print(test_malware.shape)

(51696,)


In [234]:
def preprocess_binary_filefile_path(file_bytes, s):
    l = len(file_bytes)
    padding_length = s * np.ceil(l / s) - l
    padded_bytes = np.pad(file_bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
    x = np.mean(padded_bytes.reshape(-1, int(np.ceil(l / s))), axis=1)
    x_normalized = x / 255

    return x_normalized

In [255]:
test_input = preprocess_binary_filefile_path(test_malware, 2**14)
test_input = torch.tensor(test_input)
print(test_input.shape)

torch.Size([16384])


In [256]:
test_input = test_input.unsqueeze(0).unsqueeze(1).float()

In [257]:
test_input.shape

torch.Size([1, 1, 16384])

In [259]:
model.eval()
with torch.no_grad():  # Gradiensek számításának kikapcsolása
    output = model(test_input)  # Modell alkalmazása
    pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
    pred_labels = 1 if pred_prob >= 0.5 else 0

print(f"Predicted label: {convert_label(pred_labels)}, Prob: {pred_prob}")
print("True label: ", convert_label(1))

Predicted label: malware, Prob: 0.9996621608734131
True label:  malware


In [282]:
binary = torch.tensor(test_malware).squeeze()
s = 2**14  
suffix_percentage = 0.05  

suffix_length = int(binary.shape[0] * suffix_percentage)
# Kezdetben nullákat fűzünk hozzá
# suffix_zeros = torch.zeros(suffix_length, dtype=data.dtype)
# modified_data = torch.cat((data, suffix_zeros), dim=0)

# Most meghatározzuk, mely pozíciók módosíthatóak
# Ebben a példában feltételezzük, hogy ezeket a pozíciókat előzőleg már meghatároztuk.
modifiable_positions, M_len, modified_data = determine_modifiable_positions(binary, s, suffix_length)

print(modifiable_positions)
print(M_len)

# Most random értékeket helyezünk a módosítható pozíciókra
for pos in modifiable_positions:
    # Itt egy konkrét módosítási logikát kellene implementálni, például:
    # Ha a 'modifiable_positions' valódi indexeket tartalmaz, használhatunk egy egyszerű hozzárendelést
    # Ebben a példában egyszerűség kedvéért feltételezzük, hogy az 'modifiable_positions' tartalmazza az indexeket közvetlenül
    modified_data[pos] = torch.rand(1)  # Random érték [0, 1] között

# Itt folytatódik a modell kiértékelése a módosított adatokkal, hasonlóan az előző példádhoz


{12925, 12926, 12927, 12928, 12929, 12930, 12931, 12932, 12933, 12934, 12935, 12936, 12937, 12938, 12939, 12940, 12941, 12942, 12943, 12944, 12945, 12946, 12947, 12948, 12949, 12950, 12951, 12952, 12953, 12954, 12955, 12956, 12957, 12958, 12959, 12960, 12961, 12962, 12963, 12964, 12965, 12966, 12967, 12968, 12969, 12970, 12971, 12972, 12973, 12974, 12975, 12976, 12977, 12978, 12979, 12980, 12981, 12982, 12983, 12984, 12985, 12986, 12987, 12988, 12989, 12990, 12991, 12992, 12993, 12994, 12995, 12996, 12997, 12998, 12999, 13000, 13001, 13002, 13003, 13004, 13005, 13006, 13007, 13008, 13009, 13010, 13011, 13012, 13013, 13014, 13015, 13016, 13017, 13018, 13019, 13020, 13021, 13022, 13023, 13024, 13025, 13026, 13027, 13028, 13029, 13030, 13031, 13032, 13033, 13034, 13035, 13036, 13037, 13038, 13039, 13040, 13041, 13042, 13043, 13044, 13045, 13046, 13047, 13048, 13049, 13050, 13051, 13052, 13053, 13054, 13055, 13056, 13057, 13058, 13059, 13060, 13061, 13062, 13063, 13064, 13065, 13066, 13067

In [274]:
print(binary.shape)
print(modified_data.shape)

torch.Size([51696])
torch.Size([54280])


In [283]:
modified_test_input = preprocess_binary_filefile_path(modified_data, 2**14)
modified_test_input = torch.tensor(modified_test_input)
print(modified_test_input.shape)

torch.Size([16384])


In [284]:
model.eval()
with torch.no_grad():  # Gradiensek számításának kikapcsolása
    output = model(modified_test_input.unsqueeze(0).unsqueeze(1).float())  # Modell alkalmazása
    pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
    pred_labels = 1 if pred_prob >= 0.5 else 0

print(f"Predicted label: {convert_label(pred_labels)}, Prob: {pred_prob}")
print("True label: ", convert_label(1))

Predicted label: malware, Prob: 0.9997403025627136
True label:  malware
